Import Packages,data and download stopwords.

In [79]:
import kagglehub
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
import tqdm
import pprint
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))
nltk.download('stopwords')
nltk.download('punkt')
nltk.data.path.append(r'C:/nltk_data')



fake_news = pd.read_csv(r'C:\Users\Nteit\Downloads\Fake\Fake.csv')
true_news = pd.read_csv(r'C:\Users\Nteit\Downloads\True\True.csv')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nteit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nteit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drop rows with empty news text 


In [80]:
fake_news=fake_news.drop(fake_news[fake_news['text']==' '].index,axis=0)
true_news=true_news.drop(true_news[true_news['text']==' '].index,axis=0)

Browse top 5 of entries of fake and true news dataset

In [81]:
fake_news.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [82]:
true_news.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Check for missing values

In [83]:
fake_news['text'].isnull().sum()

0

In [84]:
true_news['text'].isnull().sum()

0

Target Variable

In [85]:
fake_news['Class']=1
true_news['Class']=0

Remove the Reueters tag and back from real news

In [86]:
true_news["text"] = true_news["text"].str.replace(r".*?\(Reuters\) -", "", regex=True)

Create a unified dataframe

In [87]:
cols=fake_news.columns.values
cols=np.append(cols,'Class')
news=pd.DataFrame(columns=cols)
news=pd.concat([fake_news,true_news])

In [88]:
print(news.groupby('Class').count())

       title   text  subject   date
Class                              
0      21416  21416    21416  21416
1      22855  22855    22855  22855


Keep only text and class

In [89]:
data=news[['text','Class']]
data.head()

,text,Class
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


Lowercase

In [90]:
data['text']=data['text'].str.lower()
data = data.reset_index(drop=True)

C:\Users\Nteit\AppData\Local\Temp\ipykernel_2828\1782702889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text']=data['text'].str.lower()


Clean text

In [91]:
def clean_text(text):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
        text = re.sub(r'\[.*?\]', '', text)  # Remove text in square brackets
        text = re.sub(r"\\W", " ", text)  # Remove non-word characters
        text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
        text = re.sub(r'<.*?>+', '', text)  # Remove HTML tags
        text = re.sub(r'\w*\d\w*', '', text)  # Remove words containing numbers
        return text.strip()

In [92]:
data['text']=data['text'].apply(clean_text)


Create a copy 

In [93]:
textdata=pd.DataFrame(columns=['text','Class'])
for i in range(len(data)):
    textdata.loc[i,'text'] = data.loc[i,'text']
    textdata.loc[i,'Class']=int(data.iloc[i, data.columns.get_loc('Class')])

Tokenize

In [94]:
textdata['tokens'] = textdata['text'].apply(lambda x: x.split())

Remove Stopwords

In [95]:

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

In [96]:
textdata['tokens'] = textdata['tokens'].apply(remove_stopwords)

Stem

In [97]:
def text_stemmer(tokens):
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens
    


In [98]:
textdata['tokens'] = textdata['tokens'].apply(text_stemmer)


Shuffle

In [99]:
textdata = textdata.sample(frac=1, random_state=1).reset_index(drop=True)

Reconstruct text 

In [100]:
textdata['clean_text'] = textdata['tokens'].apply(lambda tokens: ' '.join(tokens))

Preview clean texts

In [101]:

textdata[['text','clean_text']].head()



,text,clean_text
0,military intelligence programs account for bi...,militari intellig program account billion pent...
1,us judge james robart emerged from relative ob...,us judg jame robart emerg rel obscur saturday ...
2,century wire says rich men love to gamble over...,centuri wire say rich men love gambl last week...
3,us president donald trump has invited three la...,us presid donald trump invit three latin ameri...
4,the pentagon plans to transfer about a dozen i...,pentagon plan transfer dozen inmat guantanamo ...


Drop empty rows after cleaning 

In [102]:
textdata=textdata.drop(textdata[textdata['clean_text']==''].index,axis=0)
textdata = textdata.reset_index(drop=True)

Create Models list

In [103]:

logistic_model = LogisticRegression(penalty=None)
lasso_model = LogisticRegression(penalty='l1', solver='liblinear')
ridge_model = LogisticRegression(penalty='l2')
dt_classifier = DecisionTreeClassifier(random_state=42)

models=[ MultinomialNB(),logistic_model,lasso_model,ridge_model,dt_classifier,svm.SVC()]

Run models for Binary Vectorizer and save results to a dataframe

In [104]:

vectorizers=[CountVectorizer(binary=True)]
Y = textdata['Class']
resultsdfac=pd.DataFrame()
for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    for model in models:
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        if model==lasso_model :
            resultsdfac.loc['lasso','Binary'] = accuracy_score(Y_test, Y_pred)
        elif model==ridge_model:
            resultsdfac.loc['ridge','Binary']= accuracy_score(Y_test, Y_pred)
        elif  model==logistic_model:
            resultsdfac.loc['logistic','Binary'] = accuracy_score(Y_test, Y_pred) 
        else:
            resultsdfac.loc[model.__class__.__name__,'Binary']= accuracy_score(Y_test, Y_pred) 


Run models for Counts Vectorizer,Tfidf and save results to a dataframe

In [105]:
vectorizers=[CountVectorizer(binary=False),TfidfVectorizer(max_features=5000)]


Y = textdata['Class']

for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    for model in models:
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        if model==lasso_model :
            resultsdfac.loc['lasso',vectorizer.__class__.__name__] = accuracy_score(Y_test, Y_pred)
        elif model==ridge_model:
            resultsdfac.loc['ridge',vectorizer.__class__.__name__]= accuracy_score(Y_test, Y_pred)
        elif  model==logistic_model:
            resultsdfac.loc['logistic',vectorizer.__class__.__name__] = accuracy_score(Y_test, Y_pred) 
        else:
            resultsdfac.loc[model.__class__.__name__,vectorizer.__class__.__name__]= accuracy_score(Y_test, Y_pred) 



C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Run models on word2vec with 300 dimensions

In [106]:
def get_average_word2vec(tokens_list, w2vec, vector_size=300):
  
    # Filter out words not in the Word2Vec vocabulary
    valid_words = [w2vec.wv[word] for word in tokens_list if word in w2vec.wv]

    # If no valid words, return a zero vector
    if not valid_words:
        return np.zeros(vector_size)
    
    # Calculate the mean of valid word vectors
    tmp = np.vstack(valid_words)  # Stack vectors vertically
    result = np.mean(tmp, axis=0)  # Calculate mean across rows
    return result
    # If no valid words, return a zero vector of the desired size
    
logistic_model = LogisticRegression(penalty=None)
lasso_model = LogisticRegression(penalty='l1', solver='liblinear')
ridge_model = LogisticRegression(penalty='l2')
dt_classifier = DecisionTreeClassifier(random_state=42)

models=[ logistic_model,lasso_model,ridge_model,dt_classifier,svm.SVC()]  
X_train, X_test, Y_train, Y_test = train_test_split(textdata['tokens'], Y, test_size=0.2, random_state=2)
w2vec = Word2Vec(sentences=X_train, vector_size=300, window=5, min_count=1, workers=6)



Train_trans=np.zeros((len(X_train),300))
Test_trans=np.zeros((len(X_test),300))
i=0
for idx in X_train.index :     
    Train_trans[i,:] = get_average_word2vec(X_train[idx], w2vec) 
    i=i+1
       
i=0
for idx in X_test.index :   
    
    Test_trans[i,:] = get_average_word2vec(X_test[idx], w2vec) 
    i=i+1
    
Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
for model in models:
        model.fit(Train_trans, Y_train)
        Y_pred = model.predict(Test_trans)
        if model==lasso_model :
            resultsdfac.loc['lasso','word2vec'] = accuracy_score(Y_test, Y_pred)
        elif model==ridge_model:
            resultsdfac.loc['ridge','word2vec']= accuracy_score(Y_test, Y_pred)
        elif  model==logistic_model:
            resultsdfac.loc['logistic','word2vec'] = accuracy_score(Y_test, Y_pred) 
        else:
            resultsdfac.loc[model.__class__.__name__,'word2vec']= accuracy_score(Y_test, Y_pred) 




C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [107]:
print(w2vec.wv.most_similar('trump'))


[('presidentelect', 0.5541456937789917), ('trumprush', 0.4533597528934479), ('obama', 0.4419034719467163), ('jtrump', 0.4337442219257355), ('deliveredand', 0.4319506883621216), ('mcgahnthat', 0.42233526706695557), ('actual', 0.4220310151576996), ('assit', 0.4157116711139679), ('visittrump', 0.4130549430847168), ('administrationpresidentelect', 0.4120168089866638)]


In [108]:
base_estimator = DecisionTreeClassifier(max_depth=1) 
rf_model = AdaBoostClassifier(
    base_estimator=base_estimator,
    n_estimators=50,  # Number of weak learners
    learning_rate=1.0,  # Shrinks the contribution of each weak learner
    random_state=42
)



vectorizers=[CountVectorizer(binary=True)]
Y = textdata['Class']
for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    rf_model.fit(X_train, Y_train)
    Y_pred = rf_model.predict(X_test)       
    resultsdfac.loc[rf_model.__class__.__name__,'Binary']= accuracy_score(Y_test, Y_pred) 



C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [109]:
vectorizers=[CountVectorizer(binary=False),TfidfVectorizer(max_features=5000)]


Y = textdata['Class']

for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    rf_model.fit(X_train, Y_train)
    Y_pred = rf_model.predict(X_test)       
    resultsdfac.loc[rf_model.__class__.__name__,vectorizer.__class__.__name__]= accuracy_score(Y_test, Y_pred) 


C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [110]:

X_train, X_test, Y_train, Y_test = train_test_split(textdata['tokens'], Y, test_size=0.2, random_state=2)
w2vec = Word2Vec(sentences=X_train, vector_size=300, window=5, min_count=1, workers=6)



Train_trans=np.zeros((len(X_train),300))
Test_trans=np.zeros((len(X_test),300))
i=0
for idx in X_train.index :     
    Train_trans[i,:] = get_average_word2vec(X_train[idx], w2vec) 
    i=i+1
       
i=0
for idx in X_test.index :   
    
    Test_trans[i,:] = get_average_word2vec(X_test[idx], w2vec) 
    i=i+1
    
Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
rf_model.fit(Train_trans, Y_train)
Y_pred = rf_model.predict(Test_trans)
resultsdfac.loc[rf_model.__class__.__name__,'word2vec']= accuracy_score(Y_test, Y_pred) 



C:\Users\Nteit\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [111]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

vectorizers=[CountVectorizer(binary=True)]
Y = textdata['Class']
for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    rf_model.fit(X_train, Y_train)
    Y_pred = rf_model.predict(X_test)       
    resultsdfac.loc[rf_model.__class__.__name__,'Binary']= accuracy_score(Y_test, Y_pred) 

In [112]:
vectorizers=[CountVectorizer(binary=False),TfidfVectorizer(max_features=5000)]


Y = textdata['Class']

for vectorizer in vectorizers:  
    X_train, X_test, Y_train, Y_test = train_test_split(textdata['clean_text'], Y, test_size=0.2, random_state=2)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
    Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
    rf_model.fit(X_train, Y_train)
    Y_pred = rf_model.predict(X_test)       
    resultsdfac.loc[rf_model.__class__.__name__,vectorizer.__class__.__name__]= accuracy_score(Y_test, Y_pred) 

In [113]:

X_train, X_test, Y_train, Y_test = train_test_split(textdata['tokens'], Y, test_size=0.2, random_state=2)
w2vec = Word2Vec(sentences=X_train, vector_size=300, window=5, min_count=1, workers=6)



Train_trans=np.zeros((len(X_train),300))
Test_trans=np.zeros((len(X_test),300))
i=0
for idx in X_train.index :     
    Train_trans[i,:] = get_average_word2vec(X_train[idx], w2vec) 
    i=i+1
       
i=0
for idx in X_test.index :   
    
    Test_trans[i,:] = get_average_word2vec(X_test[idx], w2vec) 
    i=i+1
    
Y_train = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_train)
Y_test = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y_test)
rf_model.fit(Train_trans, Y_train)
Y_pred = rf_model.predict(Test_trans)
resultsdfac.loc[rf_model.__class__.__name__,'word2vec']= accuracy_score(Y_test, Y_pred) 


In [114]:
print(resultsdfac)



                          Binary  CountVectorizer  TfidfVectorizer  word2vec
MultinomialNB           0.954859         0.948750         0.922050       NaN
logistic                0.987103         0.985971         0.985971  0.970811
lasso                   0.987555         0.984501         0.979636  0.970472
ridge                   0.989365         0.987103         0.979975  0.969906
DecisionTreeClassifier  0.945016         0.943093         0.936192  0.920919
SVC                     0.989478         0.980880         0.988347  0.975902
AdaBoostClassifier      0.946713         0.949316         0.948863  0.937323
RandomForestClassifier  0.974092         0.975563         0.982012  0.964136


Grid Search on Logistic Regression for Binary Vectorizer

In [115]:

log_reg = LogisticRegression(penalty='l2')
param_grid = {
    'C': loguniform(1e-4, 1e2),        # Regularization strength
    'solver': ['lbfgs', 'saga', 'newton-cg', 'sag'],  # Solvers supporting L2
    'max_iter': [100, 200, 500], # Max iterations
    'tol': [1e-5, 1e-4, 1e-3]        # Tolerance for convergence
   
}

random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_grid,
    n_iter=50,  # Number of random configurations to try
    scoring='accuracy',  # Or another appropriate metric
    cv=5,  # Number of cross-validation folds
    verbose=2,
    n_jobs=-1,  # Use all available processors
    random_state=2
)


vectorizer=CountVectorizer(binary=True)
X = vectorizer.fit_transform(textdata['clean_text'])
Y = textdata['Class']
Y = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y)




random_search.fit(X, Y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001EB2308A4D0>,
                                        'max_iter': [100, 200, 500],
                                        'solver': ['lbfgs', 'saga', 'newton-cg',
                                                   'sag'],
                                        'tol': [1e-05, 0.0001, 0.001]},
                   random_state=2, scoring='accuracy', verbose=2)

In [116]:
print("Best Parameters:", random_search.best_params_)
print("Best Accuracy:", random_search.best_score_)

Best Parameters: {'C': 0.4473556706796925, 'max_iter': 100, 'solver': 'newton-cg', 'tol': 1e-05}
Best Accuracy: 0.9880752541357308


In [117]:
model=LogisticRegression(C=0.4473556706796925, solver='newton-cg', tol=1e-05)
vectorizer=CountVectorizer(binary=True)
X = vectorizer.fit_transform(textdata['clean_text'])
Y = textdata['Class']
Y = np.vectorize(lambda x: pd.to_numeric(x, errors='coerce'))(Y)
model.fit(X, Y)




LogisticRegression(C=0.4473556706796925, solver='newton-cg', tol=1e-05)

Test

In [118]:
prompt=['BREAKING: Donald Trump Announces Plan to Colonize Mars Former. President Donald Trump unveiled an ambitious plan today, declaring his intention to lead the charge in colonizing Mars. Speaking at a rally, he stated, “No one’s ever done Mars like we’re going to do it. It’ll be tremendous, believe me.” Trump claimed his new initiative, "Trump Galactic," would establish "the biggest, most luxurious Martian city ever." Critics dismissed the plan as unrealistic, but supporters hailed it as visionary. SpaceX founder Elon Musk declined to comment, fueling speculation about potential collaboration.Stay tuned for developments on this out-of-this-world endeavor. ']
dftry=pd.DataFrame(data=prompt,columns=['text'])
print(prompt)


['BREAKING: Donald Trump Announces Plan to Colonize Mars Former. President Donald Trump unveiled an ambitious plan today, declaring his intention to lead the charge in colonizing Mars. Speaking at a rally, he stated, “No one’s ever done Mars like we’re going to do it. It’ll be tremendous, believe me.” Trump claimed his new initiative, "Trump Galactic," would establish "the biggest, most luxurious Martian city ever." Critics dismissed the plan as unrealistic, but supporters hailed it as visionary. SpaceX founder Elon Musk declined to comment, fueling speculation about potential collaboration.Stay tuned for developments on this out-of-this-world endeavor. ']


In [119]:
dftry['text']=dftry['text'].apply(clean_text)
dftry['tokens'] = dftry['text'].apply(lambda x: x.split())
dftry['tokens'] = dftry['tokens'].apply(remove_stopwords)
dftry['tokens'] = dftry['tokens'].apply(text_stemmer)
dftry['clean_text'] = dftry['tokens'].apply(lambda tokens: ' '.join(tokens))

X_test = vectorizer.transform(dftry['clean_text'])
model.predict(X_test)

array([1])

In [120]:
prompt=['A former Colorado Bureau of Investigation DNA scientist appeared in court Thursday to face criminal charges over data tampering that authorities said raises questions about the validity of more than 500 cases.Problems with the scientist’s work were found in cases involving homicide, sexual assault, robbery and other crimes, according to a law enforcement affidavit.In at least two cases, both homicides, the defendants received lesser sentences under plea deals than they could have faced if they went to trial because prosecutors were afraid Yvonne “Missy” Woods’ involvement could lead to acquittals.Woods was described as a “star analyst” by a former colleague who was interviewed by investigators, but also one who worked too fast and was “not the most thorough,” according to an internal affairs report.Authorities haven’t found any evidence of wrongful convictions, but prosecutors across the state are continuing to review the impacted cases.“This gets to the heart of whether or not science can be trusted, whether or not law enforcement can be trusted and quite frankly whether the judicial system can be trusted,” Jefferson County judge Graham Peper said during the short hearing.Woods allegedly told investigators at one point that she had changed data to complete cases more quickly, according to an arrest affidavit.Woods faces 52 counts of forgery, 48 counts of attempting to influence a public servant and one count each of perjury and cybercrime, for alleged misconduct between 2008 and 2023.The fallout from the alleged misconduct is still unfolding.In the most recent case to be impacted, Michael Shannel Jefferson was sentenced last week to 32 years in prison in the home invasion killing of Roger Dean in 1985. Jefferson was identified as a suspect in the cold case in 2021 through DNA evidence.']
print(prompt)

['A former Colorado Bureau of Investigation DNA scientist appeared in court Thursday to face criminal charges over data tampering that authorities said raises questions about the validity of more than 500 cases.Problems with the scientist’s work were found in cases involving homicide, sexual assault, robbery and other crimes, according to a law enforcement affidavit.In at least two cases, both homicides, the defendants received lesser sentences under plea deals than they could have faced if they went to trial because prosecutors were afraid Yvonne “Missy” Woods’ involvement could lead to acquittals.Woods was described as a “star analyst” by a former colleague who was interviewed by investigators, but also one who worked too fast and was “not the most thorough,” according to an internal affairs report.Authorities haven’t found any evidence of wrongful convictions, but prosecutors across the state are continuing to review the impacted cases.“This gets to the heart of whether or not scien

In [121]:
dftry=pd.DataFrame(data=prompt,columns=['text'])

dftry['text']=dftry['text'].apply(clean_text)
dftry['tokens'] = dftry['text'].apply(lambda x: x.split())
dftry['tokens'] = dftry['tokens'].apply(remove_stopwords)
dftry['tokens'] = dftry['tokens'].apply(text_stemmer)
dftry['clean_text'] = dftry['tokens'].apply(lambda tokens: ' '.join(tokens))

X_test = vectorizer.transform(dftry['clean_text'])
model.predict(X_test)

array([0])

In [122]:
prompt=['In a stunning press announcement, former President Donald Trump declared a "war on Mars," claiming the planet poses a threat to Earth.Mars has been stealing Earth’s energy for years—bad energy, folks," Trump stated, without providing evidence. He proposed creating a new military branch, the "Galactic Space Force," to counter this supposed threat.Critics dismissed the claim as science fiction, while supporters praised Trump’s "bold vision." Trump also hinted at secret backing from Elon Musk but offered no details.NASA officials, caught off guard, declined to comment, with one insider joking, "We’re focused on science, not space wars."Trump promised the initiative would be "tremendous," though experts remain skeptical.']
print(prompt)

['In a stunning press announcement, former President Donald Trump declared a "war on Mars," claiming the planet poses a threat to Earth.Mars has been stealing Earth’s energy for years—bad energy, folks," Trump stated, without providing evidence. He proposed creating a new military branch, the "Galactic Space Force," to counter this supposed threat.Critics dismissed the claim as science fiction, while supporters praised Trump’s "bold vision." Trump also hinted at secret backing from Elon Musk but offered no details.NASA officials, caught off guard, declined to comment, with one insider joking, "We’re focused on science, not space wars."Trump promised the initiative would be "tremendous," though experts remain skeptical.']


In [123]:
dftry=pd.DataFrame(data=prompt,columns=['text'])

dftry['text']=dftry['text'].apply(clean_text)
dftry['tokens'] = dftry['text'].apply(lambda x: x.split())
dftry['tokens'] = dftry['tokens'].apply(remove_stopwords)
dftry['tokens'] = dftry['tokens'].apply(text_stemmer)
dftry['clean_text'] = dftry['tokens'].apply(lambda tokens: ' '.join(tokens))

X_test = vectorizer.transform(dftry['clean_text'])
model.predict(X_test)

array([1])

In [124]:
prompt=['In a shocking revelation during a livestream, Kanye West unveiled his latest vision: creating a self-sustaining city called “Yeezy City” in the middle of the Nevada desert. The city, described by Kanye as “the future of human innovation,” will reportedly feature futuristic Yeezy-designed homes, a music production hub, and a museum dedicated entirely to Kanye’s career.Yeezy City will be a utopia where creativity knows no limits,” West stated. He also hinted at plans for a cryptocurrency called “YeezyCoin” to power the local economy.Critics have questioned the feasibility of the project, while fans have already started online petitions to move there. When asked about timelines, Kanye confidently replied, “We’re breaking ground next year. Elon’s already on board.”The announcement has sparked a social media frenzy, with many wondering if Yeezy City could become the next Silicon Valley—or just another Kanye dream.']
print(prompt)

['In a shocking revelation during a livestream, Kanye West unveiled his latest vision: creating a self-sustaining city called “Yeezy City” in the middle of the Nevada desert. The city, described by Kanye as “the future of human innovation,” will reportedly feature futuristic Yeezy-designed homes, a music production hub, and a museum dedicated entirely to Kanye’s career.Yeezy City will be a utopia where creativity knows no limits,” West stated. He also hinted at plans for a cryptocurrency called “YeezyCoin” to power the local economy.Critics have questioned the feasibility of the project, while fans have already started online petitions to move there. When asked about timelines, Kanye confidently replied, “We’re breaking ground next year. Elon’s already on board.”The announcement has sparked a social media frenzy, with many wondering if Yeezy City could become the next Silicon Valley—or just another Kanye dream.']


In [125]:
dftry['text']=dftry['text'].apply(clean_text)
dftry['tokens'] = dftry['text'].apply(lambda x: x.split())
dftry['tokens'] = dftry['tokens'].apply(remove_stopwords)
dftry['tokens'] = dftry['tokens'].apply(text_stemmer)
dftry['clean_text'] = dftry['tokens'].apply(lambda tokens: ' '.join(tokens))

X_test = vectorizer.transform(dftry['clean_text'])
model.predict(X_test)

array([1])

Check the words that decide the outcome 

In [126]:
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0] 

In [127]:
sorted_indices = np.argsort(np.abs(coefficients))[::-1]
sorted_feature_names = feature_names[sorted_indices]
sorted_coefficients = coefficients[sorted_indices]
sorted_indices = np.argsort(np.abs(coefficients))[::-1]
sorted_feature_names = feature_names[sorted_indices]
sorted_coefficients = coefficients[sorted_indices]

top_positive_words = [(sorted_feature_names[i], sorted_coefficients[i]) 
                      for i in range(10) if sorted_coefficients[i] > 0]
top_negative_words = [(sorted_feature_names[i], sorted_coefficients[i]) 
                      for i in range(10) if sorted_coefficients[i] < 0]
print("Top fake news predictors:")
for word, coef in top_positive_words:
    print(f"{word}: {coef:.4f}")
print("\nTop real news words:")
for word, coef in top_negative_words:
    print(f"{word}: {coef:.4f}")


Top fake news predictors:
via: 3.2403
sen: 1.8893
gop: 1.8139
obama: 1.8003
rep: 1.7852
imag: 1.7586

Top real news words:
said: -2.3925
thursday: -1.7609
dont: -1.7605
wednesday: -1.7304
